### Ticket Classifier and answering using ollama 3

In [9]:
import ollama
from datetime import datetime
import os

def classify_and_answer_tickets_llm(tickets, username="test_user"):
    if not tickets or len(tickets) == 0:
        print("Error: No tickets found to classify.")
        return "No data to process"

    # --- Preparing input text ---
    text_data = ""
    for idx, ticket in enumerate(tickets):
        text_data += f"[Ticket #{idx+1}]\nText: {ticket}\n\n"

    print("Input text length:", len(text_data))
    print("Sample input:", text_data[:500])

    # --- Defined system prompt ---
    system_prompt = """
You are an AI support ticket assistant.

You will receive multiple IT/HR support tickets.
For each ticket:
1. Classify it into one of these categories:
   - Login/Access Issue
   - HR Query
   - IT Support
   - Other
2. Provide a clear and helpful answer/solution.

Return results in this exact format:

Ticket: "<original ticket text>"
Category: <predicted category>
Answer: <short, helpful solution>
---
"""

    try:
        response = ollama.chat(
            model='llama3:latest',   # local model name
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": text_data[:3000]}  
            ]
        )

        # Debugging raw output
        print("Raw Ollama response:", response)

        if "message" in response:
            output = response["message"]["content"]
        elif "content" in response:
            output = response["content"]
        else:
            output = str(response)

        print("\n--- Ticket Classification + Answers ---\n")
        print(output)

        # --- Save results ---
        os.makedirs("classified_tickets", exist_ok=True)
        filename = f"classified_tickets/{username}_tickets_with_answers.txt"

        with open(filename, "w", encoding="utf-8") as f:
            f.write(output)

        return output

    except Exception as e:
        print("LLM call failed:", e)
        return "LLM error"


# --- Sample Run ---
tickets = [
    "I forgot my password, how to reset it?",
    "I can’t log in, as password is incorrect.",
    "How to see leave balance?",
    "Laptop not turning on after update."
]

if __name__ == "__main__":
    classify_and_answer_tickets_llm(tickets, username="divyank")


Input text length: 219
Sample input: [Ticket #1]
Text: I forgot my password, how to reset it?

[Ticket #2]
Text: I can’t log in, as password is incorrect.

[Ticket #3]
Text: How to see leave balance?

[Ticket #4]
Text: Laptop not turning on after update.


Raw Ollama response: model='llama3:latest' created_at='2025-09-15T21:43:09.8255204Z' done=True done_reason='stop' total_duration=92214669000 load_duration=12357113600 prompt_eval_count=171 prompt_eval_duration=3180705400 eval_count=256 eval_duration=76665210900 message=Message(role='assistant', content='---\n\nTicket: "I forgot my password, how to reset it?"\nCategory: Login/Access Issue\nAnswer: You can reset your password by clicking the \'Forgot Password\' link on our login page and following the prompts. If you\'re still having trouble, please contact our IT department for assistance.\n\n---\nTicket: "I can’t log in, as password is incorrect."\nCategory: Login/Access Issue\nAnswer: It looks like there might be a typo or mistake w

### Dataset trained and manual ticking

In [ ]:
import ollama
import pandas as pd
import os
from datetime import datetime

# --- Loading dataset ---
DATASET_PATH = "tickets.csv"
ticket_data = pd.read_csv(DATASET_PATH)

# Creating few-shot examples 
few_shot_examples = ""
for i, row in ticket_data.sample(40, random_state=42).iterrows():  # 40 random examples
    few_shot_examples += f'ticket: "{row["ticket"]}"\ncategory: {row["category"]}\n---\n'


In [ ]:
# --- Defined system prompt ---
system_prompt = f"""
You are an AI support assistant trained on IT & HR tickets.

You will receive a support ticket.
First classify it into one of these categories:
- Login/Access Issue
- HR Query
- IT Support
- Other

Then provide a short, helpful response to the ticket.

Here are some training examples:

{few_shot_examples}

Now classify the user's ticket.
Return the results in this exact format:

Ticket: "<original ticket text>"
Category: <predicted category>
Answer: <helpful short answer>
---
"""

def classify_ticket_llm(user_ticket, username="test_user"):
    try:
        response = ollama.chat(
            model="llama3:latest",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_ticket}
            ]
        )

        # Extracting response
        if "message" in response:
            output = response["message"]["content"]
        elif "content" in response:
            output = response["content"]
        else:
            output = str(response)

        # Save results
        os.makedirs("classified_tickets", exist_ok=True)
        filename = f"classified_tickets/{username}_ticket_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(output)

        return output

    except Exception as e:
        print("LLM call failed:", e)
        return "LLM error"

In [8]:
# --- Interactive loop ---
if __name__ == "__main__":
    while True:
        user_ticket = input("\nEnter your support ticket (or type 'exit' to quit): ")
        if user_ticket.lower() == "exit":
            break
        result = classify_ticket_llm(user_ticket, username="divyank")
        print("\n--- Classification & Answer ---\n")
        print(result)


--- Classification & Answer ---

---

Ticket: "brightness of my laptop not working"
Category: IT Support
Answer: Sorry to hear that! Can you please try adjusting the brightness using the keyboard shortcut (usually F5 or F11) and see if it works? If not, we can troubleshoot further.
---

--- Classification & Answer ---

Ticket: "empty coffee machine in the office"
Category: Other
Answer: "Sorry to hear that! Please report this issue to our facilities team so they can restock or repair the machine as soon as possible. In the meantime, you may want to consider alternative options for your morning cup."

--- Classification & Answer ---

Ticket: ""
Category: IT Support
Answer: Please try restarting your laptop and then check if the issue persists. If not, please feel free to provide more details so we can assist you further.
---


KeyboardInterrupt: 